# Mini Aevol OpenMP Optimization Analysis

This notebook is used to analyze the results of the Mini Aevol OpenMP optimization. 

Aevol is a digital genetics model: populations of digital organisms are subjected to a process of selection and variation, which creates a Darwinian dynamics. The simulation platform comes along with a set of tools for analysing phylogenies and measuring many characteristics of the organisms and populations along evolution.

The results can be regenerated in the `experiments` folder by running the following block.

In [ ]:
!make setup
!make run all 

## Performance

### Level 0 - No parallelization

To understand how to increase the performance of a system, it is important to understand the bottlenecks of the system. This can be done by profiling the system. The profiling results are stored in the `experiments` folder. The following block will generate the profiling results of the mini aevol with parallelization level 0 (no parallelization).

In [ ]:
!make prof L=0

This generates the following profiling results in the `Instruments` app (MacOS):

![Level 0 Profiling](./resources/PL00.png)

We can see that the program spend almost 70% of the time in the `run_step` function (total of all the calls). This makes senses due to it is the place where the mutations are happening. However, this also means that we can optimize the function using OpenMP to parellelize the operations that are executed over the individuals.

The `run_step` function is called N times in the simulation. This means that we any optimization we do in the function, will be affecting the system N times. The `run_step` function is composed of:

![Level 0 Profiling](./resources/PL01.png)



We have two possible approach to start optimizing the program:

- Analyze the section in the same order they are in the previous image and start optimizing what is possible.
- Analyze the sections with more than 1% of the total time and start optimizing what is possible in order of execution.

We are going to choose the second approach for clarity.

### Level 1 - Parallelize the mutation of the individuals

We saw, in the previous section, that the `evaluate`, `prepare_mutation`, `selection` and `apply_mutations` functions are taking almost 40% of the total time. This functions are executed for each individual inside a sequential for loop. We can parallelize this loop to improve the performance of the program. If they still are consuming more than 1% of the total time, we can optimize them them. 

This is optimization done by adding the `#pragma omp parallel for` directive in first section of the `run_step` function:

```c++
#pragma omp parallel for shared(dna_mutator_array_) if (level_ > 0) num_threads(4)
for (int indiv_id = 0; indiv_id < nb_indivs_; indiv_id++) {
    selection(indiv_id);
    prepare_mutation(indiv_id);

    if (dna_mutator_array_[indiv_id]->hasMutate()) {
        auto &mutant = internal_organisms_[indiv_id];
        mutant->apply_mutations(dna_mutator_array_[indiv_id]->mutation_list_);
        mutant->evaluate(target);
    }
}
```

We can start with a 4 threads parallelization due to it is a good number for the number of cores of the machine we are using. Later we're going to try different values to see how it affects the performance. The `if (level_ > 0)` is used to accumulate optimization on each level, this means that next levels will include this level.

The following block will generate the profiling results of the mini aevol with parallelization level 1.

In [ ]:
!make prof L=1

This generates the following profiling results in the `Instruments` app (MacOS):

![Level 0 Profiling](./resources/PL10.png)

We can see that the execution time of `run_step` decreased radically from almost 70% to almost 30%. This means that we have a 2x improvement in the performance of the program. 

Now, the runtime of each function in the `run_step` function is the following:

![Level 1 Profiling](./resources/PL11.png)

### Level 2 - Parallelize the natural selection

```c++
// Search for the best
struct Case { float value; int index; };    
#pragma omp declare reduction(best : struct Case : omp_out = omp_in.value > omp_out.value ? omp_in : omp_out)

struct Case best_fitness; 
best_fitness.value = prev_internal_organisms_[0]->fitness;
best_fitness.index = 0;

#pragma omp parallel for reduction(best:best_fitness) shared(prev_internal_organisms_, internal_organisms_) if (level_ > 1) num_threads(4)
for (int indiv_id = 1; indiv_id < nb_indivs_; indiv_id++) {
    if (prev_internal_organisms_[indiv_id]->fitness > best_fitness.value) {
        best_fitness.index = indiv_id;
        best_fitness.value = prev_internal_organisms_[indiv_id]->fitness;
    }
}
best_indiv = prev_internal_organisms_[best_fitness.index];
```

![Level 2 Profiling](./resources/PL20.png)